In [2]:
import numpy as np
import nashpy as nash
import axelrod as axl
import random
from collections import namedtuple

In [7]:
def who_is_playing(num_of_opponents, long_run_strategies=False):
    """
    A function to choose which strategies will be playing against the Defector.

    'num_of_opponents' is a numeric variable which states how many players will 
    be competing (EXCLUDING the Defector).

    'long_run_strategies' is a Boolean variable which states whether strategies 
    which have a long running time should be included in the competitors or 
    not. It has a default value of False.

    A list containing the selected strategies is returned.
    """

    if long_run_strategies == True:
        filterstrategies = {
            "manipulates_state": False,
            "manipulates_source": False,
            "inspects_source": False,
        }
    else:
        filterstrategies = {
            "long_run_time": False,
            "manipulates_state": False,
            "manipulates_source": False,
            "inspects_source": False,
        }

    filtered_strategies = axl.filtered_strategies(filterstrategies)
    filtered_strategies.remove(axl.Defector)

    opponent_strategies = random.sample(filtered_strategies, num_of_opponents)
    list_of_players = [opponent() for opponent in opponent_strategies]
    list_of_players.append(axl.Defector())

    return list_of_players

In [3]:
def probabilities_of_defection(
    tournament_repeat,
    player_list,
    probs_of_game_ending,
    nash_equilibrium_algorithm,
    set_seed, noise
):
    """
    A function which executes varying tournaments of A Prisoner's Dilemma, each 
    with a distinct probabilistic ending, and then computes the Nash Equilibria 
    of the resulting mean payoff matrix, where:

    'tournament_repeat' is a numeric variable stating how many times each 
    tournament should be played;

    'player_list' is a list containing all the strategies which are competing;

    'probs_of_game_ending' is a list (or numpy array) of values between 0 and 1 
    which state the probability of a specific game ending;

    'nash_equilibrium_algorithm' is a string containing either "Support
    Enumeration", "Vertex Enumeration" or "Lemke Howson". This indicates which
    method will be used in calculating the Nash Equilibria. WARNING - the "Lemke
    Howson" algorithm may not return all Nash Equilibria;
    
    'set_seed' is a numeric variable which ensures reproducibility if the same
    value is used; and

    'noise' is a numeric variable between 0 and 1 which indicates how much noise should be included in the tournaments.

    The output is two lists: the first and second containing the least and 
    greatest probability of defection, respectively, obtained from the Nash 
    Equilibria. 
    """

    least_prob_of_defection_in_equilibria = []
    greatest_prob_of_defection_in_equilibria = []

    for probability in probs_of_game_ending:

        axl.seed(set_seed)

        tournament = axl.Tournament(
            player_list, prob_end=probability, repetitions=tournament_repeat, noise=noise
        )

        tournament_results = tournament.play(progress_bar=False)
        mean_payoff_matrix = np.array(tournament_results.payoff_matrix)

        game = nash.Game(mean_payoff_matrix, mean_payoff_matrix.transpose())

        if nash_equilibrium_algorithm == "Support Enumeration":
            nash_equilibria = list(game.support_enumeration())
            #print(nash_equilibria)

        elif nash_equilibrium_algorithm == "Vertex Enumeration":
            nash_equilibria = list(game.vertex_enumeration())
            #print(nash_equilibria)

        elif nash_equilibrium_algorithm == "Lemke Howson":
            nash_equilibria = list(game.lemke_howson_enumeration())
            #print(nash_equilibria)

        else:
            raise Exception(
                "nash_equilibrium_algorithm should be one of ['Support Enumeration', 'Vertex Enumeration', 'Lemke Howson']"
                )

        length_of_nash_eq = len(nash_equilibria)

        prob_of_defection_in_equilibria = [
            sigma_1[-1] for sigma_1, _ in nash_equilibria
        ]

        least_prob_of_defection_in_equilibria.append(
            min(prob_of_defection_in_equilibria)
        )

        greatest_prob_of_defection_in_equilibria.append(
            max(prob_of_defection_in_equilibria)
        )

    return probs_of_game_ending, mean_payoff_matrix, tournament_repeat, length_of_nash_eq, nash_equilibria, least_prob_of_defection_in_equilibria, greatest_prob_of_defection_in_equilibria, noise


In [4]:
def array_to_string(numpy_array):

    """
    A function which converts a numpy array into a space separated string.
    """

    flattened_array = numpy_array.flatten()
    flattened_array_to_string = str(flattened_array).strip('[]')
    return flattened_array_to_string

In [5]:
read_into_sql = """
    INSERT into folk_theorem_experiment 
        (experiment_number, player_strategy_name, is_long_run_time, is_stochastic, memory_depth_of_strategy, prob_of_game_ending, payoff_matrix, num_of_repetitions, num_of_equilibria, nash_equilibria, least_prob_of_defection, greatest_prob_of_defection, noise)
    VALUES 
        (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""


Record = namedtuple('Record', 'experiment_number, player_strategy_name, is_long_run_time, is_stochastic, memory_depth_of_strategy, prob_of_game_ending, payoff_matrix, num_of_repetitions, num_of_equilibria, nash_equilibria, least_prob_of_defection, greatest_prob_of_defection, amount_of_noise')

In [19]:
def Record(experiment_number, player_strategy_name, is_long_run_time, is_stochastic, memory_depth_of_strategy, prob_of_game_ending, payoff_matrix, num_of_repetitions, num_of_equilibria, nash_equilibria, least_prob_of_defection, greatest_prob_of_defection, noise):

    """
    A function which writes the information gained from one player in the experiment to a single record in an SQL database, where:

    """
    payoff_matrix_as_string = array_to_string(mean_payoff_matrix)
    nash_eq_as_string = array_to_string(nash_equilibria)

    record = (record.experiment_number, record.player_strategy_name, record.is_long_run_time, record.is_stochastic, record.memory_depth_of_strategy, record.prob_of_game_ending, record.payoff_matrix, record.num_of_repetitions, record.num_of_equilibria, record.nash_equilibria, record.least_prob_of_defection, record.greatest_prob_of_defection, record.noise)

    connect_dbms_to_db.execute(read_into_sql, record)

In [8]:
players = who_is_playing(3)
players

[UsuallyDefects, Adaptive, Delayed AON1, Defector]

In [9]:
tournament = probabilities_of_defection(
    tournament_repeat=2,
    player_list=players,
    probs_of_game_ending=[0.02, 0.43, 0.6],
    nash_equilibrium_algorithm="Support Enumeration",
    set_seed=456, noise=0
)

C:\Users\sophi\Anaconda3\lib\site-packages\nashpy\algorithms\support_enumeration.py:196: RuntimeWarning: 
An even number of (4) equilibria was returned. This
indicates that the game is degenerate. Consider using another algorithm
to investigate.
                  
  warnings.warn(warning, RuntimeWarning)
C:\Users\sophi\Anaconda3\lib\site-packages\nashpy\algorithms\support_enumeration.py:196: RuntimeWarning: 
An even number of (4) equilibria was returned. This
indicates that the game is degenerate. Consider using another algorithm
to investigate.
                  
  warnings.warn(warning, RuntimeWarning)


In [10]:
tournament

([0.02, 0.43, 0.6], array([[1., 5., 5., 1.],
        [0., 3., 3., 0.],
        [0., 3., 3., 0.],
        [1., 5., 5., 1.]]), 2, [(array([1., 0., 0., 0.]),
   array([1., 0., 0., 0.])),
  (array([1., 0., 0., 0.]), array([0., 0., 0., 1.])),
  (array([0., 0., 0., 1.]), array([1., 0., 0., 0.])),
  (array([0., 0., 0., 1.]), array([0., 0., 0., 1.]))], [0.0, 0.0, 0.0], [1.0,
  1.0,
  1.0], 0)

In [13]:
probs_of_game_ending = tournament[0]
nash_equilibria = tournament[3]
least_prob_of_defection = tournament[4]
greatest_prob_of_defection = tournament[5]

for probability in probs_of_game_ending:

    for player in players:
        experiment_number = 1
        player_name = players[player]
        is_long_run_time = player_name.classifier['long_run_time']
        is_stochastic = player_name.classifier['stochastic']
        memory_depth_of_strategy = player_name.classifier['memory_depth']
        prob_of_game_ending = probability
        mean_payoff_matrix = tournament[1]
        num_of_repetitions = tournament[2]
        nash_equilibria = nash_equilibria[]

False

In [20]:
probs_of_game_ending = tournament[0]
probs_of_game_ending

[0.02, 0.43, 0.6]

In [17]:
player_name.classifier

{'memory_depth': inf,
 'stochastic': False,
 'makes_use_of': set(),
 'long_run_time': False,
 'inspects_source': False,
 'manipulates_source': False,
 'manipulates_state': False}

In [29]:
player = 'UsuallyDefects'
players.index(player)

ValueError: 'UsuallyDefects' is not in list